In [21]:
def calc_map(frame): #columns #display_id,clicked,prediciton
    pred_id=0
    i=1
    sum_=0
    p_k=0
    clicks=0
    display_ids=[]
    ranks=[]
    for row in frame:
        if row['display_id']==pred_id:
            if row['clicked']==1:
                #print row
                clicks+=1
                p_k=clicks*1.0/i
                sum_+=p_k
                display_ids.append(row['display_id'])
                ranks.append(i)
                
            i+=1
        else:
            pred_id=row['display_id']
            clicks=0
            p_k=0
            i=1
        
    return sum_,display_ids,ranks

In [20]:
def frame_to_lib_ffm(frame,key,field_list): #no le currenlty
    new_frame=sframe.SFrame()
    
    for column_name in frame.column_names():
        if column_name!=key:
            col=[str(field_list[column_name])+':'+str(row)+':1' for row in frame[column_name]]         
        else:
            col=frame[column_name]
        new_frame[column_name]=col
    return new_frame

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc # We're gonna be clearing memory a lot
import matplotlib.pyplot as plt
import seaborn as sns
import sframe
from sklearn.preprocessing import LabelEncoder

%matplotlib inline

p = sns.color_palette()
dir=os.getcwd()

print('# File sizes')
for f in os.listdir(dir):
    if 'zip' not in f:
        print(f.ljust(30) + str(round(os.path.getsize(dir+'/' + f) / 1000000, 2)) + 'MB')
        
#../../libffm/ffm-train -l 0.002 -k 16 -t 30 -r 0.1 -s 12 -p small_test_frame.csv small_train_frame.csv model_w_shift

# File sizes
small_test_frame.csv          47.0MB
Kag_Outbrain_2.ipynb          0.0MB
clicks_test.csv               506.0MB
documents_categories.csv      118.0MB
output                        160.0MB
.ipynb_checkpoints            0.0MB
clicks_train.csv              1486.0MB
a.out                         0.0MB
small1_test_frame.csv         70.0MB
train.csv                     3725.0MB
Kag_Outbrain.ipynb            0.0MB
small_train_libffm.csv        23.0MB
small1_train_frame.csv        70.0MB
page_views_sample.csv         454.0MB
events.csv                    1208.0MB
documents_topics.csv          339.0MB
documents_meta.csv            89.0MB
a                             0.0MB
promoted_content.csv          13.0MB
small_test_libffm.csv         25.0MB
model_w_shift                 358.0MB
nohup.out                     0.0MB
no_le_frame.csv               23.0MB
backtrace.0                   0.0MB
test.ipynb                    0.0MB
documents_entities.csv        324.0MB
myfile              

In [5]:
events=sframe.SFrame.read_csv('events.csv')
clicks_test=sframe.SFrame.read_csv('clicks_test.csv')
page_views=sframe.SFrame.read_csv('page_views_sample.csv')
clicks_train=sframe.SFrame.read_csv('clicks_train.csv')

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,int,int,int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Unable to parse line "303066,83e9ec48908c6a,968149,28799999,\N,US>VA>511"

Read 1061339 lines. Lines per second: 689849

Unable to parse line "11328496,7e8aa06b36db6a,1827718,752399961,\N,US>KY>529"

Read 12198726 lines. Lines per second: 1.85074e+06

...

Read 16032927 lines. Lines per second: 5.18149e+06

Read 47416685 lines. Lines per second: 5.87525e+06

Read 80175120 lines. Lines per second: 5.92408e+06

Finished parsing file /home/ubuntu/Notebooks/kaggle_outbrain/clicks_train.csv

Parsing completed. Parsed 87141731 lines in 14.4075 secs.

In [6]:
#take only part of the data
unique_display_ids=clicks_train['display_id'].unique()
displays=300000
sample_ids_train=unique_display_ids[:displays]
sample_ids_val=unique_display_ids[displays+1:displays*2]

In [7]:
#split
len(clicks_train)
clicks_t_1=clicks_train.filter_by(sample_ids_train,'display_id')
clicks_t_2=clicks_train.filter_by(sample_ids_val,'display_id')

In [6]:
#make joined full data
import time
t=time.time()
joined_train_clicks=clicks_train.join(events)
print time.time()-t
t=time.time()
joined_test_clicks=clicks_test.join(events)
print time.time()-t

112.029255152
43.5349359512


In [8]:
#make joined partial
joined_train_clicks=clicks_t_1.join(events)
joined_test_clicks=clicks_t_2.join(events)

In [7]:
#set features
ad_ids=joined_train_clicks['ad_id'].append(joined_test_clicks['ad_id'])
uuids=joined_train_clicks['uuid'].append(joined_test_clicks['uuid'])
#set side feauters
document_ids=joined_train_clicks['document_id'].append(joined_test_clicks['document_id'])
geo_locations=joined_train_clicks['geo_location'].append(joined_test_clicks['geo_location'])

In [8]:
#precprocess: create le for uuid and document id

le={}

t=time.time()
le['uuid']=LabelEncoder()
le['uuid'].fit(uuids)
print time.time()-t
t=time.time()

le['ad_id']=LabelEncoder()
le['ad_id'].fit(ad_ids)
print time.time()-t
t=time.time()

le['document_id']=LabelEncoder()
le['document_id'].fit(document_ids)
print time.time()-t
t=time.time()

le['geo_location']=LabelEncoder()
le['geo_location'].fit(geo_locations)

print time.time()-t

column_diffs=[]

296.839234829
67.2194120884
60.800645113
274.558499813


In [12]:
import pickle #saved at /datadrive

with open('le.pickle', 'wb') as f:
    pickle.dump(le, f, -1)

In [14]:
#convert to ints

t=time.time()
train_users=le['uuid'].transform(joined_train_clicks['uuid'])
test_users=le['uuid'].transform(joined_test_clicks['uuid'])
column_diffs.append(max(np.append(train_users,test_users)))
print time.time()-t
t=time.time()

test_ads=le['ad_id'].transform(joined_test_clicks['ad_id'])+column_diffs[0]
train_ads=le['ad_id'].transform(joined_train_clicks['ad_id'])+column_diffs[0]
column_diffs.append(max(np.append(test_ads,train_ads)))
print time.time()-t
t=time.time()
    
test_docs=le['document_id'].transform(joined_test_clicks['document_id'])+column_diffs[1]
train_docs=le['document_id'].transform(joined_train_clicks['document_id'])+column_diffs[1]
column_diffs.append(max(np.append(train_docs,test_docs)))
print time.time()-t
t=time.time()
    
test_geos=le['geo_location'].transform(joined_test_clicks['geo_location'])+column_diffs[2]
train_geos=le['geo_location'].transform(joined_train_clicks['geo_location'])+column_diffs[2]
print time.time()-t

386.906072855
98.8985040188
76.0966479778
286.676968098


In [17]:
#generate final data frames
t=time.time()
final_frame=sframe.SFrame({'uuid':train_users,'ad_id':train_ads,'clicked':joined_train_clicks['clicked']\
                         ,'document':train_docs,'geo_location':train_geos})
print time.time()-t
t=time.time()
final_test_frame=sframe.SFrame({'uuid':test_users,'ad_id':test_ads,'clicked':np.zeros(len(test_ads))\
                            ,'document':test_docs,'geo_location':test_geos})
print time.time()-t

53.1167919636
24.3882029057


In [18]:
#make validation set

t=time.time()
train_frame,val_frame=final_frame.random_split(0.8)
print time.time()-t

0.00105309486389


In [22]:
#convert to libffm
t=time.time()
lib_train_frame=frame_to_lib_ffm(train_frame,'clicked',{'clicked':-1, 'uuid':0, 'ad_id':1,'document':2,'geo_location':3})
print time.time()-t
t=time.time()
lib_val_frame=frame_to_lib_ffm(val_frame,'clicked',{'clicked':-1, 'uuid':0, 'ad_id':1,'document':2,'geo_location':3})
print time.time()-t
t=time.time()
lib_test_frame=frame_to_lib_ffm(final_test_frame,'clicked',{'clicked':-1, 'uuid':0, 'ad_id':1,'document':2,'geo_location':3})
print time.time()-t

336.194636106
87.0684099197
150.907740831


In [24]:
#no_le_frame=no_le_frame[no_le_frame['clicked','uuid', 'ad_id']]

t=time.time()
lib_train_frame['clicked','uuid','ad_id','document','geo_location'].export_csv('train_frame.csv',sep=' ',header=None)
print time.time()-t
t=time.time()
lib_val_frame['clicked','uuid','ad_id','document','geo_location'].export_csv('val_frame.csv',sep=' ',header=None)
print time.time()-t
t=time.time()
lib_test_frame['clicked','uuid','ad_id','document','geo_location'].export_csv('test_frame.csv',sep=' ',header=None)
print time.time()-t


165.542075872
44.2801558971
81.3458681107


In [20]:
output_list=pd.read_csv('output',header=None)
output_list[0]

0          0.194510
1          0.039637
2          0.297865
3          0.093103
4          0.348920
5          0.263523
6          0.061075
7          0.147666
8          0.082700
9          0.143351
10         0.159865
11         0.067970
12         0.482371
13         0.328920
14         0.060769
15         0.264720
16         0.633726
17         0.136945
18         0.038477
19         0.182926
20         0.142968
21         0.080430
22         0.047158
23         0.053466
24         0.445598
25         0.218477
26         0.313527
27         0.045086
28         0.310353
29         0.388842
             ...   
1549630    0.195922
1549631    0.299438
1549632    0.101766
1549633    0.079174
1549634    0.084952
1549635    0.511118
1549636    0.605184
1549637    0.203521
1549638    0.100028
1549639    0.131260
1549640    0.253350
1549641    0.346607
1549642    0.278563
1549643    0.053090
1549644    0.070113
1549645    0.257801
1549646    0.102616
1549647    0.327207
1549648    0.202521


In [31]:
#read prediction and parse it

# 1read output
output_list=pd.read_csv('output_val',header=None)

print len(output_list),len(clicks_test)

test_set=joined_test_clicks['display_id','ad_id','clicked']#
test_set['prediction']=output_list[0]
#join to test

# sort test set
test_set=test_set.sort(['display_id','prediction'],ascending=False)

17427743 32225162


RuntimeError: Column name clicked does not exist

In [34]:
#val_frame
#read prediction and parse it

# 1read output
output_list=pd.read_csv('output_val',header=None)

print len(output_list),len(clicks_test)

test_set=val_frame['display_id','ad_id','clicked']#
test_set['prediction']=output_list[0]
#join to test

# sort test set
test_set=test_set.sort(['display_id','prediction'],ascending=False)

17427743 32225162


RuntimeError: Column name display_id does not exist

In [28]:
#calc map
s,display_id,ranks=calc_map(test_set)#/200000
s/len(display_id)

KeyError: 'clicked'

In [29]:

first=1
f = open('submission','w')
for item in test_set:
    if first or item['display_id']<>pred_dis_id:
        f.write('\n')
        f.write(str(item['display_id'])+',')
    first=0
    pred_dis_id=item['display_id']    
    f.write(str(item['ad_id'])+' ') 
f.close()